<a href="https://colab.research.google.com/github/AnthonyTennis/lab-6-Data301/blob/main/Copy_of_DATA_301_Lab_6B_Anthony_Teciorowski.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bing Maps API

In this part of the lab, you will join the CityBikes data from the previous part of the lab with additional data that you will query from the Bing Maps API.

First, you will need to register for a Bing Maps Key. Follow the instructions [here](https://docs.microsoft.com/en-us/bingmaps/getting-started/bing-maps-dev-center-help/getting-a-bing-maps-key). You should be able to sign in with your Cal Poly account. When you get to Step 4, select:

- Key type: Basic
- Application type: Dev/Test

You will be able to make 125000 free requests with the resulting API key. This should be more than enough to complete this assignment.

We will be working with the [REST services in the Bing Maps API](https://docs.microsoft.com/en-us/bingmaps/rest-services/). Click on the link for a complete documentation of the features.

In [56]:
apikey = "ApQNieBdaaOD48C8pTRuIGyuyNIX63Sv6_rSJHo5e_ilahNOxA0tVyJ5uMfEY2Na"

## Question 1

Read in the `DataFrame` of bike stations in the United States from Part A of this lab. Restrict to the stations in the "Bay Wheels" network (with network ID "bay-wheels").

How many of these stations are in the city/county of San Francisco? 

(_Hint:_ Use the [Locations API](https://docs.microsoft.com/en-us/bingmaps/rest-services/locations/) to get the address associated with each latitude and longitude coordinate.)

In [57]:
import requests
import pandas as pd
from google.colab import drive
import os

drive.mount("/content/drive", force_remount=True)

# Read in the DataFrame of bike stations in the United States from Part A
stations_df = pd.read_csv("drive/MyDrive/stations_data.csv")
# Restrict to the stations in the "Bay Wheels" network
bay_wheels_df = stations_df[stations_df["network_name"] == "Bay Wheels"]

# Use the Locations API to reverse geocode each station and obtain the associated address
addresses = []
for index, row in bay_wheels_df.iterrows():
    latitude = row["latitude"]
    longitude = row["longitude"]
    response = requests.get(f"https://dev.virtualearth.net/REST/v1/Locations/{latitude},{longitude}", params={"key": apikey})
    data = response.json()
    address = data["resourceSets"][0]["resources"][0]["address"]["formattedAddress"]
    addresses.append(address)

Mounted at /content/drive


In [58]:
# Add the addresses to the DataFrame
bay_wheels_df["address"] = addresses

# Filter the stations located in the city/county of San Francisco
san_francisco_df = bay_wheels_df[bay_wheels_df["address"].str.contains("San Francisco")]

# Print the number of stations in San Francisco
print(f"There are {len(san_francisco_df)} stations in San Francisco.")


There are 293 stations in San Francisco.


<ipython-input-58-c367aeb81e2c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bay_wheels_df["address"] = addresses


## Question 2

You want to go to Coit Tower. To save money, you decide to ride a "Bay Wheels" bike to the closest station and hail a cab from there to Coit Tower. What station should you bike to so that you are as close to Coit Tower as possible (as measured by driving distance)? Does your answer agree with the one that you obtained in Part A of this lab? If not, why does it differ?

_Hints:_ 
- You should restrict your attention to bike stations that are in San Francisco, which you determined in Question 1. 
- Use the [Routes API](https://docs.microsoft.com/en-us/bingmaps/rest-services/routes/calculate-a-distance-matrix) to calculate a distance matrix between Coit Tower and the bike stations. 
- You can do this with just one call to the API. Because there are too many stations, it is impossible to specify all the locations in the URL. Instead, you should make a POST request (`requests.post`), passing in the parameters as a JSON object through the `json=` parameter of `requests.post`. Read the API documentation carefully to learn how to use the POST API.

In [81]:
# Define the API endpoint
url = "https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix"
apikey = "ApQNieBdaaOD48C8pTRuIGyuyNIX63Sv6_rSJHo5e_ilahNOxA0tVyJ5uMfEY2Na"

# Define the parameters for the API call
params = {
    # "key": apikey,
    "origins": [{
        "latitude": 37.802747, 
        "longitude": -122.405861
    }],  # Coit Tower coordinates
    "travelMode": "driving",
    "distanceUnit": "mi",
    "destinations": [{"latitude": row[0], "longitude": row[1]} for index, row in san_francisco_df[["latitude", "longitude"]].iterrows()],
}

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {apikey}"
}

# Make the API call
response = requests.post(url, headers=headers, json=params)

# Parse the API response
response_data = response.json()
print(params)
print(response_data)


{'origins': [{'latitude': 37.802747, 'longitude': -122.405861}], 'travelMode': 'driving', 'distanceUnit': 'mi', 'destinations': [{'latitude': 37.7630152, 'longitude': -122.4264968}, {'latitude': 37.76704457969368, 'longitude': -122.39083349704742}, {'latitude': 37.7789994, 'longitude': -122.4368608}, {'latitude': 37.78383, 'longitude': -122.39887}, {'latitude': 37.7316566899845, 'longitude': -122.45112150907516}, {'latitude': 37.80477, 'longitude': -122.403234}, {'latitude': 37.771762110313176, 'longitude': -122.39843755960464}, {'latitude': 37.771058, 'longitude': -122.402717}, {'latitude': 37.772300631747626, 'longitude': -122.3930275440216}, {'latitude': 37.7662185, 'longitude': -122.4310597}, {'latitude': 37.7440667, 'longitude': -122.4214722}, {'latitude': 37.7472996, 'longitude': -122.4114029}, {'latitude': 37.7436839, 'longitude': -122.4268059}, {'latitude': 37.745738796183325, 'longitude': -122.42214024066924}, {'latitude': 37.7478584, 'longitude': -122.4249863}, {'latitude': 3

In [69]:
print(response_data)
# Get the driving distances
distances = response_data["resourceSets"][0]["resources"][0]["results"]

# Add the driving distances to the station DataFrame
san_francisco_df["driving_distance"] = [result["travelDistance"] for result in distances]

# Find the station with the smallest driving distance
min_distance_station = san_francisco_df.loc[san_francisco_df["driving_distance"].idxmin()]

# Print the result
print(f"The station closest to Coit Tower is '{min_distance_station['name']}' with a driving distance of {min_distance_station['driving_distance']} miles.")

{'authenticationResultCode': 'NoCredentials', 'brandLogoUri': 'http://dev.virtualearth.net/Branding/logo_powered_by.png', 'copyright': 'Copyright © 2023 Microsoft and its suppliers. All rights reserved. This API cannot be accessed and the content and any results may not be used, reproduced or transmitted in any manner without express written permission from Microsoft Corporation.', 'errorDetails': ['Access was denied. You may have entered your credentials incorrectly, or you might not have access to the requested resource or operation.'], 'resourceSets': [], 'statusCode': 401, 'statusDescription': 'Unauthorized', 'traceId': 'b2f95c2f066a493498bed11fc99d5c26|BN00006424|0.0.0.0'}


IndexError: ignored

For some reason, I can not get this API to run. I have read the documentation and am still stuck. Will consult with my team about this.

## Submission Instructions

- Copy this notebook to your own Drive, if you have not already.
- Restart this notebook and run the cells from beginning to end. 
  - Go to Runtime > Restart and Run All.
- Rename this notebook by clicking on "DATA 301 Lab 6B - YOUR NAMES HERE" at the very top of this page. Replace "YOUR NAMES HERE" with the first and last names of you (and your partners, for Phase 2).
- Get the link to your notebook:
  - Click on "Share" at the top-right. 
  - Change the settings to "Anyone with the link can view". 
  - Copy the sharing link into Canvas.